In [206]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

In [207]:
emi_pred=pd.read_csv("files\\emi_prediction_dataset.csv", low_memory=False)

In [208]:
emi_pred.head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38,Female,Married,Professional,82600.0,Private,0.9,Mid-size,Rented,20000.0,...,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan EMI,850000.0,15,Not_Eligible,500.0
1,38,Female,Married,Graduate,21500.0,Private,7.0,MNC,Family,0.0,...,Yes,4100.0,714.0,92500.0,26900.0,E-commerce Shopping EMI,128000.0,19,Not_Eligible,700.0
2,38,Male,Married,Professional,86100.0,Private,5.8,Startup,Own,0.0,...,No,0.0,650.0,672100.0,324200.0,Education EMI,306000.0,16,Eligible,27775.0
3,58,Female,Married,High School,66800.0,Private,2.2,Mid-size,Own,0.0,...,No,0.0,685.0,440900.0,178100.0,Vehicle EMI,304000.0,83,Eligible,16170.0
4,48,Female,Married,Professional,57300.0,Private,3.4,Mid-size,Family,0.0,...,No,0.0,770.0,97300.0,28200.0,Home Appliances EMI,252000.0,7,Not_Eligible,500.0


In [209]:
emi_pred.shape

(404800, 27)

In [210]:
emi_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   age                     404800 non-null  object 
 1   gender                  404800 non-null  object 
 2   marital_status          404800 non-null  object 
 3   education               402396 non-null  object 
 4   monthly_salary          404800 non-null  object 
 5   employment_type         404800 non-null  object 
 6   years_of_employment     404800 non-null  float64
 7   company_type            404800 non-null  object 
 8   house_type              404800 non-null  object 
 9   monthly_rent            402374 non-null  float64
 10  family_size             404800 non-null  int64  
 11  dependents              404800 non-null  int64  
 12  school_fees             404800 non-null  float64
 13  college_fees            404800 non-null  float64
 14  travel_expenses     

In [211]:
emi_pred.columns

Index(['age', 'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'existing_loans', 'current_emi_amount',
       'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario',
       'requested_amount', 'requested_tenure', 'emi_eligibility',
       'max_monthly_emi'],
      dtype='object')

In [212]:
col_x=['age', 'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'existing_loans', 'current_emi_amount',
       'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario',
       'requested_amount', 'requested_tenure']

In [213]:
emi_pred["emi_eligibility"].value_counts()

emi_eligibility
Not_Eligible    312868
Eligible         74444
High_Risk        17488
Name: count, dtype: int64

In [214]:
emi_pred[emi_pred["emi_eligibility"]=="Not_Eligible"].head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38,Female,Married,Professional,82600.0,Private,0.9,Mid-size,Rented,20000.0,...,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan EMI,850000.0,15,Not_Eligible,500.0
1,38,Female,Married,Graduate,21500.0,Private,7.0,MNC,Family,0.0,...,Yes,4100.0,714.0,92500.0,26900.0,E-commerce Shopping EMI,128000.0,19,Not_Eligible,700.0
4,48,Female,Married,Professional,57300.0,Private,3.4,Mid-size,Family,0.0,...,No,0.0,770.0,97300.0,28200.0,Home Appliances EMI,252000.0,7,Not_Eligible,500.0
5,38,Male,Married,High School,38800.0,Government,5.0,Large Indian,Rented,13500.0,...,No,0.0,792.0,260800.0,55600.0,Vehicle EMI,1257000.0,53,Not_Eligible,500.0
6,58,Male,Single,High School,27100.0,Private,36.0,Large Indian,Own,0.0,...,No,0.0,741.0,68000.0,36200.0,Home Appliances EMI,280000.0,11,Not_Eligible,1950.0


In [215]:
emi_pred[emi_pred["emi_eligibility"]=="Eligible"].head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
2,38,Male,Married,Professional,86100.0,Private,5.8,Startup,Own,0.0,...,No,0.0,650.0,672100.0,324200.0,Education EMI,306000.0,16,Eligible,27775.0
3,58,Female,Married,High School,66800.0,Private,2.2,Mid-size,Own,0.0,...,No,0.0,685.0,440900.0,178100.0,Vehicle EMI,304000.0,83,Eligible,16170.0
12,32,male,Married,Graduate,11837.0,Private,1.3,Large Indian,Rented,11700.0,...,No,0.0,695.0,129600.0,70900.0,E-commerce Shopping EMI,30000.0,12,Eligible,13440.0
14,27,Female,Married,Post Graduate,110800.0,Government,0.5,MNC,Family,0.0,...,No,0.0,788.0,157300.0,40500.0,E-commerce Shopping EMI,83000.0,21,Eligible,27360.0
21,27,Female,Single,Post Graduate,67300.0,Government,0.8,MNC,Rented,16400.0,...,No,0.0,765.0,448600.0,142800.0,Education EMI,226000.0,36,Eligible,13002.0


In [216]:
emi_pred.isna().sum()

age                          0
gender                       0
marital_status               0
education                 2404
monthly_salary               0
employment_type              0
years_of_employment          0
company_type                 0
house_type                   0
monthly_rent              2426
family_size                  0
dependents                   0
school_fees                  0
college_fees                 0
travel_expenses              0
groceries_utilities          0
other_monthly_expenses       0
existing_loans               0
current_emi_amount           0
credit_score              2420
bank_balance              2426
emergency_fund            2351
emi_scenario                 0
requested_amount             0
requested_tenure             0
emi_eligibility              0
max_monthly_emi              0
dtype: int64

In [217]:
emi_pred[emi_pred["education"].isna()].head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
111,38,Female,Married,NaN,46100.0,Private,3.4,MNC,Own,0.0,...,No,0.0,730.0,308200.0,88800.0,Vehicle EMI,126000.0,58,Eligible,6985.0
426,38,Female,Married,NaN,15000.0,Government,4.2,MNC,Family,0.0,...,No,0.0,752.0,112200.0,55800.0,Home Appliances EMI,280000.0,12,Not_Eligible,500.0
455,48,Male,Married,NaN,53200.0,Private,3.4,Small,Rented,15800.0,...,No,0.0,630.0,298700.0,141600.0,Vehicle EMI,1000000.0,54,Not_Eligible,500.0
593,48,Female,Married,NaN,42300.0,Self-employed,15.6,Large Indian,Family,0.0,...,No,0.0,753.0,267900.0,112000.0,Home Appliances EMI,233000.0,20,High_Risk,14374.8
773,32,Female,Single,NaN,46100.0,Private,10.0,Large Indian,Rented,11400.0,...,No,0.0,735.0,68800.0,38500.0,E-commerce Shopping EMI,141000.0,7,Not_Eligible,6120.0


In [218]:
emi_pred["education"].value_counts()

education
Graduate         181015
Post Graduate    100314
High School       60732
Professional      60335
Name: count, dtype: int64

In [219]:
emi_pred[emi_pred["monthly_rent"].isna()].head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
116,38,Male,Married,Graduate,20000.0,Private,2.1,Large Indian,Own,NaN,...,No,0.0,748.0,18400.0,9500.0,Vehicle EMI,238000.0,69,Not_Eligible,500.0
183,48,Male,Married,Post Graduate,111000.0,Private,1.7,Large Indian,Rented,NaN,...,No,0.0,803.0,98300.0,29200.0,Education EMI,311000.0,24,Eligible,20160.0
226,58,Male,Married,Graduate,85400.0,Private,7.1,Large Indian,Family,NaN,...,Yes,14800.0,661.0,241900.0,59400.0,Vehicle EMI,103000.0,83,Eligible,8015.0
429,48,Male,Married,High School,49500.0,Private,26.0,Large Indian,Own,NaN,...,No,0.0,690.0,123700.0,64200.0,Education EMI,466000.0,8,Not_Eligible,15150.0
638,27,Male,Single,Graduate,34500.0,Private,1.1,Large Indian,Own,NaN,...,No,0.0,596.0,224300.0,61900.0,Home Appliances EMI,122000.0,17,High_Risk,9416.0


In [220]:
emi_pred[emi_pred["credit_score"].isna()].head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
154,27,Male,Married,Professional,63700.0,Private,4.2,MNC,Own,7917.0,...,Yes,16700.0,NaN,320700.0,173600.0,Home Appliances EMI,103000.0,7,Not_Eligible,3955.0
619,48,Female,Married,Graduate,18700.0,Government,3.6,Small,Rented,3100.0,...,Yes,3700.0,NaN,16000.0,5200.0,Vehicle EMI,1468000.0,67,Not_Eligible,862.4
693,38,Male,Married,Professional,30000.0,Private,0.5,MNC,Rented,8200.0,...,No,0.0,NaN,107900.0,31500.0,Vehicle EMI,1130000.0,19,Not_Eligible,500.0
739,38,Male,Married,Post Graduate,65900.0,Government,1.6,Mid-size,Rented,11800.0,...,No,0.0,NaN,353300.0,172800.0,Vehicle EMI,88000.0,47,Not_Eligible,650.0
796,38,Male,Married,Post Graduate,51900.0,Private,5.8,Large Indian,Family,0.0,...,No,0.0,NaN,280000.0,134200.0,Home Appliances EMI,91000.0,21,Eligible,11750.0


In [221]:
emi_pred[emi_pred["bank_balance"].isna()].head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
136,32,Female,Single,Graduate,41500.0,Private,1.1,Mid-size,Own,0.0,...,Yes,9800.0,656.0,NaN,86700.0,E-commerce Shopping EMI,129000.0,5,Not_Eligible,980.0
298,27,Female,Married,Post Graduate,92400.0,Private,0.5,Large Indian,Family,0.0,...,Yes,13200.0,677.0,NaN,275900.0,Personal Loan EMI,935000.0,41,Not_Eligible,6608.0
700,48,Male,Married,Post Graduate,114900.0,Private,20.5,MNC,Family,0.0,...,No,0.0,632.0,NaN,70600.0,Personal Loan EMI,543000.0,46,High_Risk,17088.0
1010,38,Male,Single,High School,30900.0,Private,3.4,MNC,Rented,7600.0,...,No,0.0,703.0,NaN,25100.0,Home Appliances EMI,204000.0,34,Not_Eligible,6350.0
1090,38,Male,Married,Graduate,40800.0,Government,2.2,Mid-size,Own,0.0,...,Yes,6200.0,728.0,NaN,66400.0,Vehicle EMI,1347000.0,26,Not_Eligible,6737.5


In [222]:
emi_pred[emi_pred["emergency_fund"].isna()].head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
237,38,Male,Married,Graduate,56500.0,Private,2.9,Large Indian,Rented,11500.0,...,No,0.0,676.0,325900.0,NaN,E-commerce Shopping EMI,81000.0,19,Eligible,10050.0
267,48,Male,Married,Graduate,46800.0,Private,0.6,Large Indian,Family,0.0,...,Yes,12000.0,654.0,229600.0,NaN,Personal Loan EMI,199000.0,33,Not_Eligible,500.0
347,32,Male,Married,Graduate,97500.0,Private,2.2,Startup,Own,0.0,...,No,0.0,707.0,767800.0,NaN,E-commerce Shopping EMI,95000.0,14,Eligible,22440.0
879,48,Female,Married,Graduate,36100.0,Private,4.7,MNC,Family,0.0,...,Yes,7400.0,639.0,45900.0,NaN,Home Appliances EMI,150000.0,12,Not_Eligible,2172.8
923,27,Male,Single,Post Graduate,67000.0,Government,0.5,MNC,Rented,21500.0,...,No,0.0,705.0,347900.0,NaN,E-commerce Shopping EMI,164000.0,11,Not_Eligible,10900.0


In [223]:
emi_pred.dtypes

age                        object
gender                     object
marital_status             object
education                  object
monthly_salary             object
employment_type            object
years_of_employment       float64
company_type               object
house_type                 object
monthly_rent              float64
family_size                 int64
dependents                  int64
school_fees               float64
college_fees              float64
travel_expenses           float64
groceries_utilities       float64
other_monthly_expenses    float64
existing_loans             object
current_emi_amount        float64
credit_score              float64
bank_balance               object
emergency_fund            float64
emi_scenario               object
requested_amount          float64
requested_tenure            int64
emi_eligibility            object
max_monthly_emi           float64
dtype: object

In [224]:
emi_pred[emi_pred["bank_balance"].isna()]

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
136,32,Female,Single,Graduate,41500.0,Private,1.1,Mid-size,Own,0.0,...,Yes,9800.0,656.0,NaN,86700.0,E-commerce Shopping EMI,129000.0,5,Not_Eligible,980.0
298,27,Female,Married,Post Graduate,92400.0,Private,0.5,Large Indian,Family,0.0,...,Yes,13200.0,677.0,NaN,275900.0,Personal Loan EMI,935000.0,41,Not_Eligible,6608.0
700,48,Male,Married,Post Graduate,114900.0,Private,20.5,MNC,Family,0.0,...,No,0.0,632.0,NaN,70600.0,Personal Loan EMI,543000.0,46,High_Risk,17088.0
1010,38,Male,Single,High School,30900.0,Private,3.4,MNC,Rented,7600.0,...,No,0.0,703.0,NaN,25100.0,Home Appliances EMI,204000.0,34,Not_Eligible,6350.0
1090,38,Male,Married,Graduate,40800.0,Government,2.2,Mid-size,Own,0.0,...,Yes,6200.0,728.0,NaN,66400.0,Vehicle EMI,1347000.0,26,Not_Eligible,6737.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403835,38,Male,Married,Graduate,44900.0,Private,10.6,MNC,Family,0.0,...,Yes,8100.0,808.0,NaN,121900.0,Home Appliances EMI,219000.0,22,Not_Eligible,4032.0
403944,32,Male,Single,Post Graduate,118700.0,Private,0.5,Startup,Own,0.0,...,No,0.0,746.0,NaN,106700.0,Vehicle EMI,1003000.0,76,Eligible,31640.0
404092,48,Female,Married,Professional,47400.0,Private,2.5,MNC,Own,0.0,...,Yes,7400.0,569.0,NaN,8900.0,Vehicle EMI,204000.0,68,Not_Eligible,3628.8
404495,38,Male,Married,High School,33100.0,Government,12.0,Mid-size,Own,0.0,...,No,0.0,798.0,NaN,62800.0,E-commerce Shopping EMI,132000.0,7,Not_Eligible,8712.0


In [225]:
emi_pred[pd.notna(emi_pred["bank_balance"])==False]

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
136,32,Female,Single,Graduate,41500.0,Private,1.1,Mid-size,Own,0.0,...,Yes,9800.0,656.0,NaN,86700.0,E-commerce Shopping EMI,129000.0,5,Not_Eligible,980.0
298,27,Female,Married,Post Graduate,92400.0,Private,0.5,Large Indian,Family,0.0,...,Yes,13200.0,677.0,NaN,275900.0,Personal Loan EMI,935000.0,41,Not_Eligible,6608.0
700,48,Male,Married,Post Graduate,114900.0,Private,20.5,MNC,Family,0.0,...,No,0.0,632.0,NaN,70600.0,Personal Loan EMI,543000.0,46,High_Risk,17088.0
1010,38,Male,Single,High School,30900.0,Private,3.4,MNC,Rented,7600.0,...,No,0.0,703.0,NaN,25100.0,Home Appliances EMI,204000.0,34,Not_Eligible,6350.0
1090,38,Male,Married,Graduate,40800.0,Government,2.2,Mid-size,Own,0.0,...,Yes,6200.0,728.0,NaN,66400.0,Vehicle EMI,1347000.0,26,Not_Eligible,6737.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403835,38,Male,Married,Graduate,44900.0,Private,10.6,MNC,Family,0.0,...,Yes,8100.0,808.0,NaN,121900.0,Home Appliances EMI,219000.0,22,Not_Eligible,4032.0
403944,32,Male,Single,Post Graduate,118700.0,Private,0.5,Startup,Own,0.0,...,No,0.0,746.0,NaN,106700.0,Vehicle EMI,1003000.0,76,Eligible,31640.0
404092,48,Female,Married,Professional,47400.0,Private,2.5,MNC,Own,0.0,...,Yes,7400.0,569.0,NaN,8900.0,Vehicle EMI,204000.0,68,Not_Eligible,3628.8
404495,38,Male,Married,High School,33100.0,Government,12.0,Mid-size,Own,0.0,...,No,0.0,798.0,NaN,62800.0,E-commerce Shopping EMI,132000.0,7,Not_Eligible,8712.0


In [226]:
def type_corrector(x):
    # Skip if x is NaN
    if pd.isna(x):
        return np.nan
    # Convert safely to string and handle decimals
    try:
        return int(float(str(x).split(".")[0]))
    except ValueError:
        return np.nan

In [227]:
# Montlhly salary should be in int or float but showing in object type. So converting it to int type. there are multiple ". ." observed in the data so removing that also.

emi_pred["monthly_salary"] = emi_pred["monthly_salary"].apply(type_corrector)

In [228]:
emi_pred["bank_balance"] = emi_pred["bank_balance"].apply(type_corrector)

In [229]:
emi_pred["age"] = emi_pred["age"].apply(type_corrector)

In [230]:
emi_pred.dtypes

age                         int64
gender                     object
marital_status             object
education                  object
monthly_salary              int64
employment_type            object
years_of_employment       float64
company_type               object
house_type                 object
monthly_rent              float64
family_size                 int64
dependents                  int64
school_fees               float64
college_fees              float64
travel_expenses           float64
groceries_utilities       float64
other_monthly_expenses    float64
existing_loans             object
current_emi_amount        float64
credit_score              float64
bank_balance              float64
emergency_fund            float64
emi_scenario               object
requested_amount          float64
requested_tenure            int64
emi_eligibility            object
max_monthly_emi           float64
dtype: object

In [231]:
x_data=emi_pred[col_x].copy()

In [232]:
emi_pred

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38,Female,Married,Professional,82600,Private,0.9,Mid-size,Rented,20000.0,...,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan EMI,850000.0,15,Not_Eligible,500.0
1,38,Female,Married,Graduate,21500,Private,7.0,MNC,Family,0.0,...,Yes,4100.0,714.0,92500.0,26900.0,E-commerce Shopping EMI,128000.0,19,Not_Eligible,700.0
2,38,Male,Married,Professional,86100,Private,5.8,Startup,Own,0.0,...,No,0.0,650.0,672100.0,324200.0,Education EMI,306000.0,16,Eligible,27775.0
3,58,Female,Married,High School,66800,Private,2.2,Mid-size,Own,0.0,...,No,0.0,685.0,440900.0,178100.0,Vehicle EMI,304000.0,83,Eligible,16170.0
4,48,Female,Married,Professional,57300,Private,3.4,Mid-size,Family,0.0,...,No,0.0,770.0,97300.0,28200.0,Home Appliances EMI,252000.0,7,Not_Eligible,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404795,27,Male,Married,Graduate,32400,Private,5.0,Large Indian,Rented,10400.0,...,No,0.0,649.0,62000.0,32600.0,Personal Loan EMI,506000.0,47,Not_Eligible,500.0
404796,38,Male,Married,Post Graduate,49200,Private,1.9,MNC,Own,0.0,...,No,0.0,712.0,142200.0,38100.0,Personal Loan EMI,708000.0,33,Not_Eligible,5200.0
404797,32,Male,Single,Graduate,25700,Private,3.2,MNC,Rented,6300.0,...,No,0.0,676.0,191600.0,39700.0,Home Appliances EMI,93000.0,21,High_Risk,5665.0
404798,48,Male,Married,Graduate,47200,Private,3.0,MNC,Own,0.0,...,No,0.0,784.0,170400.0,45600.0,Home Appliances EMI,144000.0,36,Eligible,14460.0


In [233]:
# Formatting string columns to have consistent capitalization and removing leading/trailing spaces
x_data[x_data.select_dtypes(include=['object']).columns]=x_data[x_data.select_dtypes(include=['object']).columns].apply(lambda x: x.str.strip().str.title())

In [234]:
x_data.select_dtypes(include=['object']).columns

Index(['gender', 'marital_status', 'education', 'employment_type',
       'company_type', 'house_type', 'existing_loans', 'emi_scenario'],
      dtype='object')

In [235]:
numeric_cols=x_data.select_dtypes(include=['int64', 'float64']).columns

In [236]:
x_data.nunique()

age                          15
gender                        4
marital_status                2
education                     4
monthly_salary            12822
employment_type               3
years_of_employment         356
company_type                  5
house_type                    3
monthly_rent               4396
family_size                   5
dependents                    5
school_fees                 132
college_fees                202
travel_expenses             284
groceries_utilities         544
other_monthly_expenses      373
existing_loans                2
current_emi_amount          508
credit_score                427
bank_balance              10623
emergency_fund             5486
emi_scenario                  5
requested_amount           1491
requested_tenure             82
dtype: int64

In [237]:
x_data.replace({"existing_loans":{"Yes":1, "No":0}}, inplace=True)

In [238]:
x_data.replace({"marital_status":{"Married":1, "Single":0}}, inplace=True)

In [239]:
x_data["gender"].value_counts()

gender
Male      241107
Female    160679
M           1843
F           1171
Name: count, dtype: int64

In [240]:
x_data.replace({"gender":{"M":1, "F":0, "Male":1, "Female":0}}, inplace=True)

In [241]:
x_data["gender"].value_counts()

gender
1    242950
0    161850
Name: count, dtype: int64

In [242]:
x_data["emi_scenario"].value_counts()

emi_scenario
Home Appliances Emi        80988
Personal Loan Emi          80980
E-Commerce Shopping Emi    80948
Education Emi              80942
Vehicle Emi                80942
Name: count, dtype: int64